In [1]:
import tensorflow as tf

In [2]:
loaded_model = tf.saved_model.load('/content/movinet-model-final')

In [4]:
import cv2

def get_frames(video_path, duration=5):
    frames = []
    capture = cv2.VideoCapture(video_path)
    fps = capture.get(cv2.CAP_PROP_FPS)
    num_frames_to_read = int(fps * duration)
    capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

    for _ in range(num_frames_to_read):
        ret, frame = capture.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (224,224))
        frames.append(frame)

    capture.release()
    return np.array(frames)/255

In [3]:
import numpy as np

In [5]:
path = "/content/agqphdxmwt.mp4"

test = get_frames(path)
test = [test]
test = np.array(test,dtype = np.float32)
test.shape

(1, 149, 224, 224, 3)

In [6]:
test = tf.convert_to_tensor(test)

In [7]:
concrete_func = loaded_model.signatures['serving_default']

# Call the concrete function with the input tensor
output_dict = concrete_func(image=test)

In [8]:
logits = output_dict['classifier_head_1']
logits = np.array(logits).reshape(-1)

probabilities = tf.nn.softmax(logits)
probabilities = probabilities.numpy()
print("Predicted probabilities:", probabilities)

Predicted probabilities: [0.33427495 0.66572505]
